In [1]:
Sys.setenv(LANG="en")
#library(Seurat)
#library(future)
#library(SpatialPCA)
library(ggplot2)
library(nicheDE)

Warning message in .recacheSubclasses(def@className, def, env):
“undefined subclass "dgCMatrix" of class "AnyMatrix"; definition not updated”


In [6]:

cd <- read.csv('intestine_for_nicheDE/counts.csv', row.names = 1)
pos <- read.csv('intestine_for_nicheDE/coord.csv', row.names = 1)
celltype <- read.csv('intestine_for_nicheDE/deconv.csv', row.names = 1)
library <- read.csv('intestine_for_nicheDE/library_avgExp.csv',row.names=1)



In [7]:
#pos <- annot[, c('x', 'y')]
#location = as.matrix(pos) 
vignette_counts= as.matrix(cd)
vignette_coord = pos 
vignette_library_matrix = as.matrix(library)
vignette_deconv_mat=as.matrix(celltype)

In [8]:
vignette_counts[1:3,1:3]
vignette_coord[1:3,1:2]
vignette_library_matrix[1:3,1:3]
vignette_deconv_mat[1:3,1:3]

,Acsl1,Acta2,Ada
cell1,0,0,0
cell2,0,39,0
cell3,0,49,0


,imagerow,imagecol
,<dbl>,<dbl>
cell1,2070.405,19.02381
cell2,2071.304,87.07246
cell3,2091.343,61.80000


,Xkr4,Gm19938,Sox17
BZE,0.006153846,0.0,0.009230769
Blood_vasc,0.000000000,0.0,1.121212125
Glial,0.100000001,0.4,0.000000000


,BZE,Blood_vasc,Glial
cell1,0,0,0
cell2,0,0,0
cell3,0,0,0


In [9]:
dim(cd)
dim(pos)
dim(library)
dim(celltype)


[1] 7367  241

[1] 7367    2

[1]    17 21197

[1] 7367   17

In [10]:
# part 2a  
#make Niche-DE object
NDE_obj = CreateNicheDEObject(vignette_counts,vignette_coord,
                              vignette_library_matrix,vignette_deconv_mat,
                              sigma = c(1,100,250))

[1] "Creating Niche-DE object"
[1] "Niche-DE object created with 7367 observations, 203 genes, 1 batch(es), and 17 cell types."


In [11]:
# part 2b 
# effective niche calculation
NDE_obj = CalculateEffectiveNiche(NDE_obj)
#NDE_obj = CalculateEffectiveNicheLargeScale(NDE_obj,batch_size = 1000, cutoff = 0.05)

[1] "Calculating effective niche for kernel bandwith 1(1 out of 3 values)."
[1] "Calculating effective niche for kernel bandwith 100(2 out of 3 values)."
[1] "Calculating effective niche for kernel bandwith 250(3 out of 3 values)."
[1] "Effective niche calculated"


In [12]:
# part 2c1 Merging and Filtering Niche-DE Objects


#print to see how many observations our niche-DE object currently has
print(NDE_obj)
#make a copy
NDE_obj_copy = NDE_obj
#merge our niche-DE object with the copy
NDE_merged = MergeObjects(list(A = NDE_obj,B = NDE_obj_copy))
#print to see that we have double the observations
print(NDE_merged)

[1] "Niche-DE object with 7367 observations, 203 genes, 1 batch(es), and 17 cell types."

[1] "Niche-DE object with 14734 observations, 203 genes, 2 batch(es), and 17 cell types."

In [13]:
# part 2c2 Filtering objects 
#get first 100 spot names
cell_names = NDE_obj@cell_names[1:100]
#filter to only include these spots
NDE_obj_filtered = Filter_NDE(NDE_obj,cell_names = cell_names)
#print to see that our new object only has 100 observations
print(NDE_obj_filtered)

[1] "Niche-DE object with 100 observations, 203 genes, 1 batch(es), and 17 cell types."

In [14]:
# part 3 for large datasets 
#Perform Niche-DE
NDE_obj = niche_DE(NDE_obj,num_cores = 4,outfile = "",C = 150, 
                   M = 10, gamma = 0.8,print = T, Int = T, batch = T,self_EN = F,G = 1)

[1] "Starting Niche-DE analysis with parameters C = 150, M = 10, gamma = 0.8."
[1] "Performing Niche-DE analysis with kernel bandwidth:1 (number 1 out of 3 values)"
[1] "Running Niche-DE in parallel"
[1] "Splitting Data into 1 chunks in order to avoid memory overload. Each chunk is less than 1 gigabytes."
[1] "Initializing cluster"
[1] "Evaluating chunk 1 out of 1"


Warning message in e$fun(obj, substitute(ex), parent.frame(), e$data):
“already exporting variable(s): constant_param, niche_DE_core, counts_chunk”


[1] "Closing cluster"
[1] "Cleaning disk for next iteration"
[1] "Performing Niche-DE analysis with kernel bandwidth:100 (number 2 out of 3 values)"
[1] "Running Niche-DE in parallel"
[1] "Splitting Data into 1 chunks in order to avoid memory overload. Each chunk is less than 1 gigabytes."
[1] "Initializing cluster"
[1] "Evaluating chunk 1 out of 1"


Warning message in e$fun(obj, substitute(ex), parent.frame(), e$data):
“already exporting variable(s): constant_param, niche_DE_core, counts_chunk”


[1] "Closing cluster"
[1] "Cleaning disk for next iteration"
[1] "Performing Niche-DE analysis with kernel bandwidth:250 (number 3 out of 3 values)"
[1] "Running Niche-DE in parallel"
[1] "Splitting Data into 1 chunks in order to avoid memory overload. Each chunk is less than 1 gigabytes."
[1] "Initializing cluster"
[1] "Evaluating chunk 1 out of 1"


Warning message in e$fun(obj, substitute(ex), parent.frame(), e$data):
“already exporting variable(s): constant_param, niche_DE_core, counts_chunk”


[1] "Closing cluster"
[1] "Cleaning disk for next iteration"
[1] "Computing Positive Niche-DE Pvalues"
[1] "Computing Gene Level Pvalues"
[1] "Combining Gene Level Pvalues Across Kernel Bandwidths"
[1] "Computing Cell Type Level Pvalues"
[1] "Combining Cell Type  Level Pvalues Across Kernel Bandwidths"
[1] "Computing and Combining interaction Level Pvalues Across Kernel bandwidths"
[1] "Computing Negative Niche-DE Pvalues"
[1] "Computing Gene Level Pvalues"
[1] "Combining Gene Level Pvalues Across Kernel Bandwidths"
[1] "Computing Cell Type Level Pvalues"
[1] "Combining Cell Type  Level Pvalues Across Kernel Bandwidths"
[1] "Computing and Combining interaction Level Pvalues Across Kernel bandwidths"
[1] "Niche-DE analysis complete. Number of Genes with niche-DE T-stat equal to 131"


Warning message in niche_DE(NDE_obj, num_cores = 4, outfile = "", C = 150, M = 10, :
“Less than 1000 genes pass. This could be due to insufficient read depth of data or size of C parameter. Consider changing choice of C parameter”


In [ ]:
#outfile: The file path to the txt on which workers will write status 
#reports on
#C: The minimum total expression of a gene across observations needed 
# for the niche-DE model to run. The default value is 150.

# M: Minimum number of spots containing the index cell type with the niche cell 
#type in its effective niche for (index,niche) niche patterns to be investigated. 
# The default value is 10

# Gamma: Percentile a gene needs to be with respect to expression in the index cell 
# type in order for the model to investigate niche patterns for that gene in the index cell. 
# The default value is 0.8 (80th percentile)

# print: Should the progress of niche-DE be tracked? If True, progress updates will be
# tracked in the outfile of the cluster. Default is True.

# Int: Logical for if data is count data. If True, a negative binomial regression is performed. 
# Otherwise linear regression with a gene specific variance is applied. Default is True.

# Batch: Logical if an indicator should be included for each batch in the regression. Default is True.

# self_EN: Logical if niche interactions between the same cell types should be considered. 
# Default is False.

# G: Number of gigabytes each core should hold. If the counts matrix is bigger than G gigabytes,
# it is split into multiple chunks such that the size of each chunk is less than G gigabytes.

In [26]:
slotNames(NDE_obj)

[1] "counts"            "coord"             "sigma"            
 [4] "num_cells"         "effective_niche"   "ref_expr"         
 [7] "cell_names"        "cell_types"        "gene_names"       
[10] "batch_ID"          "spot_distance"     "niche_DE"         
[13] "niche_DE_pval_pos" "niche_DE_pval_neg" "scale"            
[16] "Int"

In [30]:
summary(NDE_obj@niche_DE)

    Length Class  Mode
1   203    -none- list
100 203    -none- list
250 203    -none- list

In [35]:
head(NDE_obj@niche_DE[[1]])

$Fzd7
$Fzd7$T_stat
[1] NA

$Fzd7$betas
[1] NA

$Fzd7$Varcov
[1] NA

$Fzd7$nulls
  [1]   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
 [19]  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
 [37]  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
 [55]  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
 [73]  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
 [91]  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
[109] 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
[127] 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
[145] 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
[163] 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
[181] 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
[199] 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
[217] 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
[235] 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252
[253] 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270
[271] 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288
[289] 289

$Fzd7$valid
[1] 0

$Fzd7$log_likelihood
[1] NA


$Icos
$Icos$T_stat
[1] NA

$Icos$betas
[1] NA

$Icos$Varcov
[1] NA

$Icos$nulls
  [1]   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
 [19]  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
 [37]  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
 [55]  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
 [73]  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
 [91]  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
[109] 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
[127] 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
[145] 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
[163] 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
[181] 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
[199] 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
[217] 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
[235] 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252
[253] 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270
[271] 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288
[289] 289

$Icos$valid
[1] 0

$Icos$log_likelihood
[1] NA


$Fzd5
$Fzd5$T_stat
[1] NA

$Fzd5$betas
[1] NA

$Fzd5$Varcov
[1] NA

$Fzd5$nulls
  [1]   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
 [19]  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
 [37]  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
 [55]  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
 [73]  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
 [91]  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
[109] 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
[127] 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
[145] 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
[163] 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
[181] 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
[199] 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
[217] 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
[235] 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252
[253] 253 254 255 256 257 258 259 260 261 262 263 26

In [42]:
NDE_obj@niche_DE[[2]]$Fzd7$T_stat

,n: BZE,n: Blood_vasc,n: Glial,n: Goblet,n: Lymphatic,n: MZE,n: Macrophage,n: Paneth,n: Plasma,n: Rest_B,n: Stem_TA,n: Stroma,n: T_cell,n: TZE,n: Tuft,n: cDC_monocyte,n: neurons_enteroendocrine
i: BZE,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
i: Blood_vasc,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
i: Glial,NA,0.9449836,NA,NA,0.8345458,NA,0.1778618,NA,NA,NA,NA,1.009511,NA,NA,NA,NA,NA
i: Goblet,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
i: Lymphatic,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
i: MZE,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
i: Macrophage,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
i: Paneth,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
i: Plasma,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
i: Rest_B,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [53]:
write.csv(NDE_obj@niche_DE[[2]]$Fzd7$T_stat, 'Fzd7_tstat.csv')
write.csv(NDE_obj@niche_DE[[2]]$Fzd7$betas, 'Fzd7_betas.csv')
#write.csv(NDE_obj@niche_DE[[2]]$Fzd7$Varcov, 'Fzd7_varcov.csv')

dim(NDE_obj@niche_DE[[2]]$Fzd7$Varcov)
NDE_obj@niche_DE[[2]]$Fzd7$valid
dim(NDE_obj@niche_DE[[2]]$Fzd7$nulls)
NDE_obj@niche_DE[[2]]$Fzd7$log_likelihood

[1] 38 38

[1] 1

NULL

[1] -5280.284

In [62]:
write.csv(NDE_obj@niche_DE[[2]]$Apob$T_stat, 'Apob_tstat.csv')
write.csv(NDE_obj@niche_DE[[2]]$Apob$betas, 'Apob_betas.csv')
#write.csv(NDE_obj@niche_DE[[2]]$Fzd7$Varcov, 'Fzd7_varcov.csv')

#dim(NDE_obj@niche_DE[[2]]$Fzd7$Varcov)
#NDE_obj@niche_DE[[2]]$Fzd7$valid
#dim(NDE_obj@niche_DE[[2]]$Fzd7$nulls)
#NDE_obj@niche_DE[[2]]$Fzd7$log_likelihood

In [61]:
write.csv(NDE_obj@niche_DE[[2]]$Lgr5$T_stat, 'Lgr5_tstat.csv')
write.csv(NDE_obj@niche_DE[[2]]$Lgr5$betas, 'Lgr5_betas.csv')
#write.csv(NDE_obj@niche_DE[[2]]$Fzd7$Varcov, 'Fzd7_varcov.csv')

dim(NDE_obj@niche_DE[[2]]$Lgr5$Varcov)
NDE_obj@niche_DE[[2]]$Lgr5$valid
dim(NDE_obj@niche_DE[[2]]$Lgr5$nulls)
NDE_obj@niche_DE[[2]]$Lgr5$log_likelihood

[1] 38 38

[1] 1

NULL

[1] -3605.365

In [82]:
write.csv(NDE_obj@niche_DE[[2]]$Mptx2$T_stat, 'Mptx2_tstat.csv')
write.csv(NDE_obj@niche_DE[[2]]$Mptx2$betas, 'Mptx2_betas.csv')

write.csv(NDE_obj@niche_DE[[2]]$Slc12a2$T_stat, 'Slc12a2_tstat.csv')
write.csv(NDE_obj@niche_DE[[2]]$Slc12a2$betas, 'Slc12a2_betas.csv')

write.csv(NDE_obj@niche_DE[[2]]$Maoa$T_stat, 'Maoa_tstat.csv')
write.csv(NDE_obj@niche_DE[[2]]$Maoa$betas, 'Maoa_betas.csv')

write.csv(NDE_obj@niche_DE[[2]]$Clca3b$T_stat, 'Clca3b_tstat.csv')
write.csv(NDE_obj@niche_DE[[2]]$Clca3b$betas, 'Clca3b_betas.csv')

write.csv(NDE_obj@niche_DE[[2]]$Stmn1$T_stat, 'Stmn1_tstat.csv')
write.csv(NDE_obj@niche_DE[[2]]$Stmn1$betas, 'Stmn1_betas.csv')

write.csv(NDE_obj@niche_DE[[2]]$H2.Eb1$T_stat, 'H2.Eb1_tstat.csv')
write.csv(NDE_obj@niche_DE[[2]]$H2.Eb1$betas, 'H2.Eb1_betas.csv')

write.csv(NDE_obj@niche_DE[[2]]$Sox4$T_stat, 'Sox4_tstat.csv')
write.csv(NDE_obj@niche_DE[[2]]$Sox4$betas, 'Sox4_betas.csv')


In [85]:
write.csv(NDE_obj@niche_DE[[1]]$Wnt3$T_stat, 'Wnt3_tstat_100.csv')
write.csv(NDE_obj@niche_DE[[1]]$Wnt3$betas, 'Wnt3_betas_100.csv')

write.csv(NDE_obj@niche_DE$`1`$Wnt3$T_stat, 'Wnt3_tstat_1.csv')
write.csv(NDE_obj@niche_DE$`1`$Wnt3$betas, 'Wnt3_betas_1.csv')

#e command NDE_obj@niche_DE$`100`$NOC2L. Un
#write.csv(NDE_obj@niche_DE[[2]]$Fzd2$T_stat, 'Fzd2_tstat.csv')
#write.csv(NDE_obj@niche_DE[[2]]$Fzd2$betas, 'Fzd2_betas.csv')

In [54]:
#head(NDE_obj@niche_DE[[2]])

In [81]:
NDE_obj@niche_DE_pval_pos[4]

$<NA>
NULL

In [63]:
summary(NDE_obj@niche_DE_pval_pos)

                  Length Class  Mode   
gene_level          203  -none- numeric
cell_type_level    3451  -none- numeric
interaction_level 58667  -none- numeric

In [72]:
write.csv(NDE_obj@niche_DE_pval_pos$cell_type_level, 'pval_pos_cell_type_level.csv')
write.csv(NDE_obj@niche_DE_pval_neg$cell_type_level, 'pval_neg_cell_type_level.csv')

In [68]:
dim(NDE_obj@niche_DE_pval_pos$interaction_level)

[1]  17  17 203

In [70]:
dim(NDE_obj@niche_DE_pval_pos$cell_type_level)

[1] 203  17

In [71]:
dim(NDE_obj@niche_DE_pval_neg$cell_type_level)

[1] 203  17

In [55]:
# This has cell by cell type dimesnon 
# NDE_obj@effective_niche

In [18]:
# part 4 Downstream Analysis 

# 4a Getting Niche-DE Genes

DE_genes = get_niche_DE_genes(NDE_obj,'CT',index='Stem_TA',niche = 'Paneth',positive = T,alpha = 0.05)
head(DE_genes)

Warning message in get_niche_DE_genes(NDE_obj, "CT", index = "Stem_TA", niche = "Paneth", :
“Less than 30 observations containing collocalization of Stem_TA and Paneth at kernel bandwidth 1. Results may be unreliable.”


[1] "Returning Niche-DE Genes"


,Genes,Pvalues.Cell.Type
,<chr>,<dbl>
1,Mptx2,0.000000e+00
10,Slc12a2,0.000000e+00
12,Maoa,4.184506e-11
2,Clca3b,1.292967e-09
5,Apob,1.769396e-06
3,Stmn1,2.118994e-04


In [77]:
#Load enichr package
library(enrichR)
#get fibroblast tumor niche genes
fibro_tum_pos = get_niche_DE_genes(NDE_obj,'CT',index='Stem_TA',niche = 'Paneth',positive = T,alpha = 0.05)
#run pathway enrichment analysis
fibro_tum_processes = enrichr(fibro_tum_pos[,1],databases = 'Reactome_2016')
#View processes in a table
View(fibro_tum_processes$Reactome_2016)

Welcome to enrichR
Checking connection ... 

Enrichr ... 
Connection is Live!

FlyEnrichr ... 
Connection is Live!

WormEnrichr ... 
Connection is Live!

YeastEnrichr ... 
Connection is Live!

FishEnrichr ... 
Connection is Live!

OxEnrichr ... 
Connection is Live!

Warning message in get_niche_DE_genes(NDE_obj, "CT", index = "Stem_TA", niche = "Paneth", :
“Less than 30 observations containing collocalization of Stem_TA and Paneth at kernel bandwidth 1. Results may be unreliable.”


[1] "Returning Niche-DE Genes"
Uploading data to Enrichr... Done.
  Querying Reactome_2016... Done.
Parsing results... Done.


Term,Overlap,P.value,Adjusted.P.value,Old.P.value,Old.Adjusted.P.value,Odds.Ratio,Combined.Score,Genes
<chr>,<chr>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<chr>
Scavenging by Class B Receptors Homo sapiens R-HSA-3000471,1/5,0.002996653,0.04599075,0,0,454.181818,2638.9141193,APOB
Scavenging by Class F Receptors Homo sapiens R-HSA-3000484,1/6,0.003594998,0.04599075,0,0,363.327273,2044.8828145,APOB
LDL-mediated lipid transport Homo sapiens R-HSA-171052,1/6,0.003594998,0.04599075,0,0,363.327273,2044.8828145,APOB
Cation-coupled Chloride cotransporters Homo sapiens R-HSA-426117,1/7,0.004193015,0.04599075,0,0,302.757576,1657.3964566,SLC12A2
Neurotransmitter Clearance In The Synaptic Cleft Homo sapiens R-HSA-112311,1/8,0.004790703,0.04599075,0,0,259.493506,1385.9750833,MAOA
Platelet sensitization by LDL Homo sapiens R-HSA-432142,1/17,0.010155126,0.06049929,0,0,113.477273,520.8353408,APOB
Chylomicron-mediated lipid transport Homo sapiens R-HSA-174800,1/17,0.010155126,0.06049929,0,0,113.477273,520.8353408,APOB
Norepinephrine Neurotransmitter Release Cycle Homo sapiens R-HSA-181430,1/18,0.010749535,0.06049929,0,0,106.796791,484.0984099,MAOA
Scavenging by Class A Receptors Homo sapiens R-HSA-3000480,1/19,0.011343616,0.06049929,0,0,100.858586,451.7557081,APOB


In [19]:
DE_genes = get_niche_DE_genes(NDE_obj,'CT',index='Paneth',niche = 'Stem_TA',positive = T,alpha = 0.05)
head(DE_genes)

Warning message in get_niche_DE_genes(NDE_obj, "CT", index = "Paneth", niche = "Stem_TA", :
“Less than 30 observations containing collocalization of Paneth and Stem_TA at kernel bandwidth 1. Results may be unreliable.”


[1] "Returning Niche-DE Genes"


,Genes,Pvalues.Cell.Type
,<chr>,<dbl>
1,Stmn1,0.001630006
2,H2.Eb1,0.006712662
4,Maoa,0.010024284
3,Slc12a2,0.013374732


In [76]:
DE_genes = get_niche_DE_genes(NDE_obj,'I',index='Paneth',niche = 'Stem_TA',positive = F,alpha = 0.05)
head(DE_genes)

Warning message in get_niche_DE_genes(NDE_obj, "I", index = "Paneth", niche = "Stem_TA", :
“Less than 30 observations containing collocalization of Paneth and Stem_TA at kernel bandwidth 1. Results may be unreliable.”


[1] "No Niche-DE Genes at this Resolution"


genes,pval
<chr>,<dbl>


In [58]:
# 4b Finding Marker Genes in an Index Cell Type Between Two Niche Cell Types

#get marker genes
fibro_tum_markers = niche_DE_markers(NDE_obj,index = 'Stem_TA',niche1='Paneth',niche2='Goblet',0.05)
#preview output
head(fibro_tum_markers)

[1] "Finding Niche-DE marker genes in index cell type Stem_TA with niche cell type Paneth relative to niche cell type Goblet. BH procedure performed at level 0.05."


Warning message in niche_DE_markers(NDE_obj, index = "Stem_TA", niche1 = "Paneth", :
“Less than 30 observations containing collocalization of Stem_TA and Paneth at kernel bandwidth 1. Results may be unreliable.”
Warning message in niche_DE_markers(NDE_obj, index = "Stem_TA", niche1 = "Paneth", :
“Less than 30 observations containing collocalization of Stem_TA and Goblet at kernel bandwidth 1. Results may be unreliable.”


[1] "Performing Contrast test on kernel 1 out of 3"
[1] "Performing Contrast test on kernel 2 out of3"
[1] "Performing Contrast test on kernel 3 out of3"
[1] "Marker gene analysis complete."


,Genes,Adj.Pvalues
,<chr>,<dbl>
1,Fzd7,8.614817e-03
2,Mptx2,0.000000e+00
3,Lgr5,4.413913e-02
4,Sox4,9.766280e-06
5,Slc12a2,5.640174e-04


In [60]:
data("niche_net_ligand_target_matrix")
data("ramilowski_ligand_receptor_list")
fibro_tumor_LR = niche_LR_spot(NDE_obj,ligand_cell = 'Stem_TA',receptor_cell = 'Paneth',
ligand_target_matrix = niche_net_ligand_target_matrix,
lr_mat = ramilowski_ligand_receptor_list,K = 25,M = 50,alpha = 0.05,truncation_value = 3)
#preview output
head(fibro_tumor_LR)

[1] "Performing niche-LR with hyperparameters K = 25, M = 50, alpha = 0.05, truncation value = 3."
[1] "Finding ligand receptors between ligand expressing cell type Stem_TA and receptor expressing cell type Paneth."


Warning message in niche_LR_spot(NDE_obj, ligand_cell = "Stem_TA", receptor_cell = "Paneth", :
“Less than 30 observations containing collocalization of Paneth and Stem_TA at kernel bandwidth 1. Results may be unreliable.”


[1] "Calculating ligand potential scores"


ERROR: Error in if (ligand %in% colnames(L)) {: argument is of length zero
